In [8]:
import pandas as pd
racism_word = pd.read_csv('racism_word.csv')
GAB = pd.read_csv('gab_data/GabHateCorpus_annotations.tsv', encoding='utf-8',sep='\t')


In [9]:
racist_word_list = racism_word['Content']

In [10]:
racist_word_list = [i.lower() for i in racist_word_list]

In [17]:
import re
from tqdm import tqdm
import emoji

class Word_Preprocessing():
    def eliminate_url(self,df,target):
        print('Start eliminate url: : )')
        df_temp = df
        target_column_name = target
        text = df_temp[target_column_name]
        for i in tqdm(text):
            urls = re.findall(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', i)
            for i in urls:
                df_temp[target_column_name] = df_temp[target_column_name].apply(lambda x: x.replace(i, ""))
        return df_temp
    
    def eliminate_username(self,df,target):
        print('Start eliminate username: : )')
        df_temp = df
        target_column_name = target
        for i in tqdm(df_temp[target_column_name]):
            user_name = re.findall(r'@\w*', i)
            for i in user_name:
                df_temp[target_column_name] = df_temp[target_column_name].apply(lambda x: x.replace(i, ""))
        return df_temp
     
    def eliminate_hashtag(self, df, target):
        df_temp = df
        target_column_name = target
        for i in tqdm(df_temp[target_column_name]):
            user_name = re.findall(r'#\w*', i)
            for i in user_name:
                df_temp[target_column_name] = df_temp[target_column_name].apply(lambda x: x.replace(i, ""))
        return df_temp
        
    
    def eliminate_symbol(self,df,target):
        print('Start eliminate symbol: : )')
        df_temp = df
        target_column_name = target
        symbol_list = [',',"'",'!','@','$','%','^','&','*','(',')','-','+','?','>','<','=','.',':',';','  ','  ','   ','    ','      ','      ','  ']
        for i in tqdm(symbol_list):
            df_temp[target_column_name] = df_temp[target_column_name].apply(lambda x: x.replace(i, ' '))
        return df_temp
    
    def process_all(self, df,target):
        df_temp = df
        target_column_name = target
        df_remove_url = self.eliminate_url(df_temp,target_column_name)
        df_eliminate_hashtag = self.eliminate_hashtag(df_remove_url, target_column_name)
        df_remove_username = self.eliminate_username(df_eliminate_hashtag, target_column_name)
        df_remove_symbol = self.eliminate_symbol(df_remove_username, target_column_name)
        print("finished!!")
        return df_remove_symbol

In [18]:
word_preprocesser = Word_Preprocessing()

In [19]:
GAB = word_preprocesser.process_all(GAB, 'Text')

  0%|          | 28/86529 [00:00<05:57, 242.21it/s]

Start eliminate url: : )


  0%|          | 123/86529 [00:00<01:50, 781.68it/s]

Start eliminate username: : )


 11%|█         | 3/27 [00:00<00:00, 29.87it/s]

Start eliminate symbol: : )


100%|██████████| 27/27 [00:01<00:00, 26.32it/s]

finished!!


In [20]:
import numpy as np
def find_hate_sentence(df):
    hate_sentence = []
    for sentence in tqdm(df):
        for word in racist_word_list:
            if sentence.find(word)>0:
                hate_sentence.append(sentence)
    return list(np.unique(hate_sentence))

In [21]:
new_gab = find_hate_sentence(GAB['Text'])

100%|██████████| 86529/86529 [00:09<00:00, 9334.91it/s]


In [22]:
len(new_gab)

4899

In [24]:
len(GAB)

86529

In [ ]:
# new_gab is the filtered gab data based on rasicm word table
# this new table should be divided in a specific ratio and then add to crosponding
# train/dev/test attack comment tables and train again